自然言語の扱い
---------------
- 自然言語を扱う以上単語を数値に変換し、なんらかのベクトル表現する必要がある
    - これを「自然言語の分散表現、または素性」という

### Bag of Words(BOW)
- 自然言語の分散表現のうち「文章」の分散表現でもっとも簡単なものに「Bag of Words」という表現がある
- BOWでは「どの単語が何回出現したか」という数で表す

### TF-IDF
- 単純に回数だけで表現するには荒すぎるため、単語に重要度を重み付けする手法
- 「同一文章に多く出てくる単語ほどその文章を特徴付ける」「その単語が様々な他の文章にも出現する場合、重要度を下げる」というルールを用いる
    - TF(Term Frequency)は単語の出現頻度を表す
    - IDF(Inverse Document Frequency)は単語が出現する文章頻度の逆数の対数を表す
- 次元数が多くなってしまう

### LSA(Latent Semantic Analysis)
- 文章 × 単語数のBOWの行列を特異値分解したのち低次元圧縮することで文章や単語を任意のn次元に圧縮する
- 特定のタスクではBOWやTF-IDF以上の性能を発揮した
- ベクトルのそれぞれの値が何を指しているか不明のため上手く行かなかった場合の原因が不明

### PLSA(Probabilistic Latent Semantic Analysis), LDA(LatentDirichletAllocation)
- トピックモデルと言われる
- トピックモデルでは文章や単語はその背後にトピックという隠れ変数をもっており、そのトピックの分布により確率的に生起されるという考え方
    - トピックとは政治、スポーツのようなジャンルのようなもの

### ニューラル言語モデル
- Word2Vecが有名
- ニューラルネットワークへの入力はデータセットの語彙数と同じ次元のワンホットベクトルになる
    - ※ ワンホットベクトルとは、ベクトルの内1箇所が1で残りが全て0のベクトル
- 単語の特徴をうまく捉えて分散表現ができる
- 訓練が非常に速い

Word2Vec
-------------------
- 青空文庫に収録されている小説の本文データを使用する
- 分散表現をTensorBordのEmbeddingsで可視化してみる

### データの前処理

------------------

### MeCab
- 日本語の形態素解析ライブラリ
- 形態素解析とは、文章を単語に分割して、各単語の品詞や活用形の解析を行うこと
- 英語などのアルファベットベースの言語ではスペースが区切りのため必要ないが日本語では必要

### MacでMecabをインストール

- brewでmecabをインストール

```
$ brew install mecab
$ brew install mecab-ipadic
```

- pythonのライブラリをインストール

```
$ pip install mecab-python3
または
$ pip install natto-py
```

- natto-pyは、FFI(Foreign function interface)というあるプログラム言語から他のプログラムを呼び出すための仕組み

#### 新語の辞書
https://github.com/neologd/mecab-ipadic-neologd/blob/master/README.ja.md


### データの前処理
- https://github.com/thinkitcojp/TensorFlowDL-samples/blob/master/word2vec/data_set.py

In [1]:
import glob
import re
import collections
import random

import numpy as np
from natto import MeCab

class DataSet(object):

    def __init__(self, data_dir, max_vocab):

        #全データセットのファイルパスを取得
        file_pathes = []
        for file_path in glob.glob(data_dir+'*'):
            file_pathes.append(file_path)

        #ファイルを読み込み
        row_documents = [self._read_docment(file_path) for file_path in file_pathes]
        #必要な部分だけ抽出
        documents = [self._preprocessing(document) for document in row_documents]
        #形態素解析
        splited_documents = [self._morphological(document) for document in documents]

        # 1, 1の行列をベクトルに変換
        words = []
        for word_list in splited_documents:
            words.extend(word_list)
        
        #データセット作成
        self.id_sequence, self.word_frequency, self.w_to_id, self.id_to_w = self._build_data_sets(words, max_vocab)
        print('Most common words (+UNK)', self.word_frequency[:5])
        print('Sample data.')
        print(self.id_sequence[:10])
        print([self.id_to_w[i] for i in self.id_sequence[:10]])
        self.data_index = 0
        
    # ファイルの読み込み
    def _read_docment(self, file_path):
        print(file_path)
        with open(file_path, 'r', encoding='sjis') as f:
            return f.read()
       
    
    #ヘッダなどの不要データを前処理。必要な部分だけを返す。
    def _preprocessing(self, document):

        lines = document.splitlines()
        processed_line = []

        horizontal_count = 0

        for line in lines:
            #ヘッダーは読み飛ばす
            # ------- が2回出現するまではヘッダー文章なのでスキップ
            if horizontal_count < 2:
                if line.startswith('-------'):
                    horizontal_count += 1
                continue
            #フッターに入る行になったらそれ以降は無視
            if line.startswith('底本：'):
                break

            line = re.sub(r'《.*》', '', line) #ルビを除去
            line = re.sub(r'［.*］', '', line) #脚注を除去
            line =re.sub(r'[!-~]', '', line) #半角記号を除去
            line =re.sub(r'[︰-＠]', '', line) #全角記号を除去
            line = re.sub('｜', '', line) # 脚注の始まりを除去

            processed_line.append(line)

        return ''.join(processed_line)
    
    
    # 形態素解析
    # documentはstring
    def _morphological(self, document):

        word_list = []
        # MeCabの形態素解析結果のフォーマット
        # %mが形態素のそのままの形 %f[o]が品詞、%f[1]が品詞の補足情報
        # %f[6]が基本形
        with MeCab('-F%f[0],%f[1],%f[6]') as mcb:
            for token in mcb.parse(document, as_nodes=True):
                features = token.feature.split(',')
                #名詞（一般）動詞（自立）、形容詞（自立）以外は除外
                if features[0] == '名詞' and features[1] == '一般' and features[2] != '':
                    word_list.append(features[2])
                if features[0] == '動詞' and features[1] == '自立' and features[2] != '':
                    word_list.append(features[2])
                if features[0] == '形容詞' and features[1] == '自立' and features[2] != '':
                    word_list.append(features[2])
        return word_list
    
    # 辞書作成
    # max_vocab: Max Vocablary size
    def _build_data_sets(self, words, max_vocab):

        #単語出現回数を解析。出現数が少ないたんをUnknown wordとしてひとくくりに扱う
        word_frequency = [['UNW', -1]]
        # collections.Counterで単語の個数を計算
        # max_vocab - 1はUnknown wordで一つ使っているので-1している
        word_frequency.extend(collections.Counter(words).most_common(max_vocab - 1))
        #単語=>IDの辞書
        w_to_id = dict()
        for word, _ in word_frequency:
            # 単語に対して連番を振ってID管理している
            w_to_id[word] = len(w_to_id)
        #形態素解析した文章を単語IDの並びに変換
        id_sequence = list()
        unw_count = 0
        for word in words:
            #UNK処理
            if word in w_to_id:
                index = w_to_id[word]
            else:
                print("------------------------")
                index = 0
                unw_count += 1
            id_sequence.append(index)
           
        word_frequency[0][1] = unw_count
        #単語ID=>単語の辞書
        # zipでkeyとvalueを入れ替える
        id_to_w = dict(zip(w_to_id.values(), w_to_id.keys()))
        return id_sequence, word_frequency, w_to_id, id_to_w
    
    # num_skip:１つの入力をどれだけ再利用するか
    # skip_window: 左右何語までを正解対象にするか
    def create_next_batch(self, batch_size, num_skips, skip_window):

        assert batch_size % num_skips == 0
        #一つの入力の再利用回数が対象範囲全件を超えてはならない
        assert num_skips <= 2 * skip_window
        inputs = np.ndarray(shape=(batch_size), dtype=np.int32)
        labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

        span = 2 * skip_window + 1
        buffer = collections.deque(maxlen=span)
        #データセットが1週しそうならindexを最初にもどす
        if self.data_index + span > len(self.id_sequence):
            self.data_index = 0
        #初期のqueueを構築(window内の単語をすべて格納)
        buffer.extend(self.id_sequence[self.data_index:self.data_index+span])
        self.data_index += span
        
        # "//" は切り捨て除算
        for i in range(batch_size // num_skips):
            #中心は先に正解データから除外
            target = skip_window
            targets_to_avoid = [skip_window]
            for j in range(num_skips):
                #すでに選ばれている物以外から正解データのインデックスを取得
                while target in targets_to_avoid:
                    target = random.randint(0, span - 1)
                #次回以降targetにならないように
                targets_to_avoid.append(target)
                #入力値になるのはbufferの中心
                inputs[i * num_skips + j] = buffer[skip_window]
                #ランダムに指定した周辺単語が正解データに
                labels[i * num_skips + j, 0] = buffer[target]

            #次に入れる単語がデータセットにない場合はbufferには最初の値を入力
            if self.data_index == len(self.id_sequence):
                buffer = self.id_sequence[:span]
                self.data_index = span
            else:
                #bufferに次の単語を追加してindexを1進める
                buffer.append(self.id_sequence[self.data_index])
                self.data_index += 1
        #最後の方のデータが使われないことを避けるために少しだけindexを元に戻す
        self.data_index = (self.data_index + len(self.id_sequence) - span) % len(self.id_sequence)

        return inputs, labels

### pythonメモ

#### extend(リストに別のリストを追加)

`<list>.extend(<list)`

#### counter#most_common

- カウンタの中で最も多い順にリストをソート結果のリストを返す
- 引数nは返却要素数

```
>>> Counter('abracadabra').most_common(3)  
[('a', 5), ('r', 2), ('b', 2)]
```

#### string.splitlines()
- 改行文字を区切りに配列に変換

### Word2Vecの実装

In [2]:
import tensorflow as tf
import math

/Users/amoro/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# プログラムに引数を与えるためにFlagsを定義する
# 深層学習ではハイパーパラメーターを多く設定する必要があるため、ある程度多くなった場合は予めフラグ定義する
# 実行時に -- help でこのフラグの説明をみることができる
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('data_dir', 'data/', "Data set directory.")
tf.app.flags.DEFINE_string('log_dir', 'logs/', "Log directory.")
tf.app.flags.DEFINE_integer('max_vocab', 2000, "Max Vocablary size.")
tf.app.flags.DEFINE_integer('skip_window', 2, "How many words to consider left and right.")
tf.app.flags.DEFINE_integer('num_skips', 4,"How many times to reuse an input to generate a label.")
tf.app.flags.DEFINE_integer('embedding_size', 64, "Dimension of the embedding vector.")
tf.app.flags.DEFINE_integer('num_sumpled', 64, "Number of negative examples to sample." )
tf.app.flags.DEFINE_integer('num_step', 10000, "Train step." )
tf.app.flags.DEFINE_integer('batch_size', 64, "Batch size." )
tf.app.flags.DEFINE_float('learning_rate', 0.1, "Learning rate." )
tf.app.flags.DEFINE_bool('create_tsv', True, "Create words.tsv or not." )

In [4]:
def main(args):

    #データセットオブジェクトを作成
    data = DataSet(FLAGS.data_dir, FLAGS.max_vocab)

    #Embeddingsように使うラベルをtsv形式で保存
    if FLAGS.create_tsv:
        sorted_dict = sorted(data.w_to_id.items(), key=lambda x: x[1])
        words = ["{word}\n".format(word=x[0]) for x in sorted_dict]
        with open(FLAGS.log_dir+"words.tsv", 'w', encoding="utf-8") as f:
            f.writelines(words)
        print("Embeddings metadata was saved to "+FLAGS.log_dir+"/words.tsv")

    batch_size = FLAGS.batch_size
    embedding_size = FLAGS.embedding_size
    vocab_size = len(data.w_to_id)
    #placeholderの定義
    inputs = tf.placeholder(tf.int32, shape=[batch_size])
    correct = tf.placeholder(tf.int32, shape=[batch_size, 1])

    word_embedding = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),name='word_embedding')
    embed = tf.nn.embedding_lookup(word_embedding, inputs)
    w_out = tf.Variable(tf.truncated_normal([vocab_size, embedding_size], stddev =1.0 / math.sqrt(embedding_size)))
    b_out = tf.Variable(tf.zeros([vocab_size]))

    # NCE誤差
    nce_loss = tf.nn.nce_loss(weights=w_out, biases = b_out, labels=correct, inputs=embed, num_sampled=FLAGS.num_sumpled, num_classes=vocab_size)
    loss = tf.reduce_mean(nce_loss)

    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = tf.train.GradientDescentOptimizer(FLAGS.learning_rate).minimize(loss, global_step=global_step)

    init = tf.global_variables_initializer()
    saver = tf.train.Saver(max_to_keep=3)

    with tf.Session() as sess:

        ckpt_state = tf.train.get_checkpoint_state(FLAGS.log_dir)
        if ckpt_state:
            last_model = ckpt_state.model_checkpoint_path
            saver.restore(sess,last_model)
            print("model was loaded:", last_model)
        else:
            sess.run(init)
            print("initialized.")

        last_step = sess.run(global_step)
        average_loss = 0
        for i in range(FLAGS.num_step):

            step = last_step + i + 1
            batch_inputs, batch_labels = data.create_next_batch(batch_size, FLAGS.num_skips, FLAGS.skip_window)
            feed_dict = {inputs: batch_inputs, correct: batch_labels}

            _, loss_val = sess.run([train_op, loss], feed_dict=feed_dict)
            average_loss += loss_val

            if step % 100 == 0:
                average_loss /= 100
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0
                saver.save(sess, FLAGS.log_dir+'my_model.ckpt', step)

if __name__ == "__main__":
    tf.app.run()


data/bocchan.txt
Most common words (+UNK) [['UNW', 0], ('する', 204), ('云う', 120), ('ない', 61), ('なる', 58)]
Sample data.
[451, 261, 168, 262, 263, 169, 47, 452, 453, 454]
['親', '譲る', '次', '抜かす', '飛ぶ', '見せる', '答える', '親類', '西洋', 'ナイフ']
Embeddings metadata was saved to logs//words.tsv
INFO:tensorflow:Restoring parameters from logs/my_model.ckpt-10000
model was loaded: logs/my_model.ckpt-10000
Average loss at step  10100 :  120.43417282104492
Average loss at step  10200 :  87.1020263671875
Average loss at step  10300 :  63.39378364562988
Average loss at step  10400 :  50.92065855026245
Average loss at step  10500 :  41.15275133132935
Average loss at step  10600 :  33.15552888870239
Average loss at step  10700 :  27.618165712356568
Average loss at step  10800 :  23.12107080459595
Average loss at step  10900 :  20.82576331138611
Average loss at step  11000 :  17.48447808742523
Average loss at step  11100 :  16.411893615722658
Average loss at step  11200 :  13.832959871292115
Average loss at ste

SystemExit: 

/Users/amoro/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [21]:
%ls ./data/

t10k-images-idx3-ubyte.gz   train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz   train-labels-idx1-ubyte.gz
